<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
from pandas_datareader import data as web
import fix_yahoo_finance as yf

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [2]:
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2018,9,24)
tickers_df = web.get_data_yahoo(["MS","GS","JPM","C"],start,end)
tickers_df = pd.DataFrame(tickers_df['Close'])
tickers_df.head()

Symbols,C,GS,JPM,MS
Date,,,,
2013-01-02,41.250000,131.660004,44.660000,19.620001
2013-01-03,41.389999,130.940002,44.570000,19.580000
2013-01-04,42.430000,134.509995,45.360001,20.190001
2013-01-07,42.470001,134.259995,45.410000,19.799999
2013-01-08,42.459999,133.050003,45.500000,19.650000


<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
rets = tickers_df.pct_change()
rets_corr = rets.corr()
rets_corr.head()

Symbols,C,GS,JPM,MS
Symbols,,,,
C,1.000000,0.792072,0.849616,0.809965
GS,0.792072,1.000000,0.813291,0.844330
JPM,0.849616,0.813291,1.000000,0.801243
MS,0.809965,0.844330,0.801243,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
pairs = rets[['JPM','C']]
pairs.head()

Symbols,JPM,C
Date,,
2013-01-02,NaN,NaN
2013-01-03,-0.002015,0.003394
2013-01-04,0.017725,0.025127
2013-01-07,0.001102,0.000943
2013-01-08,0.001982,-0.000236


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
new_df = rets[['C','JPM']].iloc[1:-1]
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']
new_df.describe()

Symbols,C,JPM
count,1442.000000,1442.000000
mean,0.000514,0.000749
std,0.014853,0.012829
min,-0.093567,-0.069477
25%,-0.007238,-0.005664
50%,0.000384,0.000488
75%,0.008250,0.007658
max,0.073185,0.083286


<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [7]:
a = c_mean+(c_std*0.25)
b = j_mean-(j_std*0.25)
new_df['Long_JPM_Short_C'] = np.where((new_df['C']>a) & (new_df['JPM']<b),True,False)
new_df.head()

Symbols,C,JPM,Long_JPM_Short_C
Date,,,
2013-01-03,0.003394,-0.002015,False
2013-01-04,0.025127,0.017725,False
2013-01-07,0.000943,0.001102,False
2013-01-08,-0.000236,0.001982,False
2013-01-09,-0.009892,-0.000659,False


<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [8]:
x = j_mean+(0.25*j_std)
z = c_mean-(0.25*c_std)
new_df['Long_C_Short_JPM'] = np.where((new_df['JPM']>x)&(new_df['C']<z),True, False)
new_df.head()

Symbols,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM
Date,,,,
2013-01-03,0.003394,-0.002015,False,False
2013-01-04,0.025127,0.017725,False,False
2013-01-07,0.000943,0.001102,False,False
2013-01-08,-0.000236,0.001982,False,False
2013-01-09,-0.009892,-0.000659,False,False


<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [9]:
k = rets['JPM'].fillna(0)
m = rets['C'].fillna(0)

new_df['Trade_return'] = 0
new_df.loc[((new_df['Long_JPM_Short_C']==True)&(new_df['Long_C_Short_JPM']==False)),'Trade_return']=k-m
new_df.loc[((new_df['Long_JPM_Short_C']==False)&(new_df['Long_C_Short_JPM']==True)),'Trade_return']=m-k
new_df.describe()

Symbols,C,JPM,Trade_return
count,1442.000000,1442.000000,1442.000000
mean,0.000514,0.000749,-0.000471
std,0.014853,0.012829,0.003246
min,-0.093567,-0.069477,-0.049712
25%,-0.007238,-0.005664,0.000000
50%,0.000384,0.000488,0.000000
75%,0.008250,0.007658,0.000000
max,0.073185,0.083286,0.000000


<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [10]:
Total = sum(new_df['Trade_return'])
print(Total)

-0.6793516982785628


Nope, no riches (negative return). I will stay in school.